# Korrigera Simmarhistorik

Utmanaren 2.0-tävling i Swimify är hårdkodad att använda Utmanaren 2.0-tider i Simmarhistoriken, vilket göra att anpassade tider och nivåer inte tillämpas.

En wg2exp-fil är en zippad JSON-fil. Packa upp JSON-filen så blir den tillgänglig att modifiera enligt egendefinierade tidsgränser. Uppdatering av nya erhållna detaljer från tävling kan göras från Lenex-filen (XML) som exporteras för Tempus efter tävlingen.

In [122]:
import json, datetime
import xmltodict

In [123]:
with open('../data/Lenex2_Resultatfil_Kronobergscupen_2023_09_16_UTF8.lef') as fd:
    lenex_data = xmltodict.parse(fd.read())

with open('../data/Simmarhistorik.json') as fd:
    json_data = json.load(fd)

swimmers = json_data['Swimmers']

In [124]:
events_yellow = {
    '50m Freestyle': ['00:49.00', '00:44.00', '00:40.00'], 
    '50m Breaststroke': ['00:59.00', '00:55.00', '00:51.00'], 
    '50m Backstroke': ['00:55.00', '00:50.00', '00:46.00'], 
    '25m Butterfly': ['00:28.00', '00:24.00', '00:21.00']
}

events_red = {
    '100m Individual Medley': ['01:35.00', '01:29.00', '01:24.00'],
    '100m Freestyle': ['01:24.00', '01:18.00', '01:13.00'],
    '100m Breaststroke': ['01:46.00', '01:39.00', '01:33.00'], 
    '100m Backstroke': ['01:36.00', '01:30.00', '01:25.00'], 
    '50m Butterfly': ['00:46.00', '00:42.00', '00:38.00']
}

In [125]:
def get_time_from_string(time_as_string):
    return datetime.datetime.strptime(time_as_string, '%M:%S.%f').time()

In [126]:
def get_lenex_athlete(license, first_name, last_name):
    clubs = lenex_data['LENEX']['MEETS']['MEET']['CLUBS']['CLUB']
    lenex_athlete = None

    for club in clubs:
        if lenex_athlete != None:
            break

        athletes = club['ATHLETES']['ATHLETE']
        for athlete in athletes:
            if (license != ''):
                if (athlete['@license'] == license):
                    lenex_athlete = athlete
                    break
            else:
                if (athlete['@firstname'] == first_name and athlete['@lastname'] == last_name):
                    lenex_athlete = athlete
                    break

    return lenex_athlete

In [127]:
def get_lenex_time(lenex_athlete, swimify_event_id):
    # No results for swimmer, return
    if 'RESULTS' not in lenex_athlete.keys():
         return None

    # Swimify events and corresponding events in Lenex
    event_map = { 
        '1020': ['1001', '1002'], 
        '1002': ['1003', '1004'], 
        '1003': ['1005', '1006'], 
        '1016': ['1007', '1008'], 
        '1017': ['1009', '1010'], 
        '1013': ['1011', '1012'],
        '1014': ['1013', '1014'],
        '1009': ['1015', '1016'],
        '1010': ['1017', '1018']
    }

    lenex_event_ids = event_map[str(swimify_event_id)]

    # Look for event result in Lenex
    lenex_result = None

    if type(lenex_athlete['RESULTS']['RESULT']) == dict:
        result = lenex_athlete['RESULTS']['RESULT']
        if (result['@eventid'] == lenex_event_ids[0]) or (result['@eventid'] == lenex_event_ids[1]):
                lenex_result = result

    if type(lenex_athlete['RESULTS']['RESULT']) == list:
        for result in lenex_athlete['RESULTS']['RESULT']:
            if (result['@eventid'] == lenex_event_ids[0]) or (result['@eventid'] == lenex_event_ids[1]):
                lenex_result = result                
                break
            
    if lenex_result == None:
        return None
    
    lenex_time = lenex_result['@swimtime'][3:] # Original format is 00:01:12.00
    return lenex_time

In [128]:
def set_kc_medal(medal, swimmer):
    kc_alvesta_date = '2023-09-16T13:00:00.0000000' # TODO: Hardcoded for KC Alvesta
    lenex_time = None
    lenex_athlete = get_lenex_athlete(swimmer['License'], swimmer['FirstName'], swimmer['LastName'])
    if lenex_athlete != None:
        lenex_time = get_lenex_time(lenex_athlete, medal['EventId'])

    if (medal['Time'] == '00:00.00' and lenex_time != None):
        # print(f'{medal["EventName"]} time is 00:00:00, look for Lenex result')
        medal['Time'] = lenex_time
        medal['TimeValue'] = int(lenex_time.replace(':', '').replace('.', '') + '0')
        medal['DateReceived'] = kc_alvesta_date # Will be reset to blank if not good enough
    
    # No history time, no lenex time, return
    if (medal['Time'] == '00:00.00'):
        return medal

    medal_times = []
    if (medal['EventName'] in events_yellow.keys()):
        medal_times = events_yellow[medal['EventName']]

    if (medal['EventName'] in events_red.keys()):
        medal_times = events_red[medal['EventName']]

    medal_time = get_time_from_string('00:00.00')
    if medal['MedalNumber'] == 10:
        medal_time = get_time_from_string(medal_times[0])
    if medal['MedalNumber'] == 20:
        medal_time = get_time_from_string(medal_times[1])
    if medal['MedalNumber'] == 30:
        medal_time = get_time_from_string(medal_times[2])

    history_time = get_time_from_string(medal['Time'])
    if (history_time <= medal_time):
        medal['HasReceived'] = True
    else:
        medal['HasReceived'] = False
        medal['DateReceived'] = '0001-01-01T00:00:00'
        
        if (lenex_time != None):
            if (get_time_from_string(lenex_time) < history_time):
                medal['Time'] = lenex_time
                medal['TimeValue'] = int(lenex_time.replace(':', '').replace('.', '') + '0')
            
            if (get_time_from_string(lenex_time) <= medal_time):
                medal['DateReceived'] = kc_alvesta_date
                medal['HasReceived'] = True

    return medal

In [129]:
def set_swimmer_medals(swimmer):
    kc_medals = []
    kc_medals_move_from_green = []
    kc_medals_move_from_green_has_been_appended = False

    for medal in swimmer['Medals']:
        if ('EventName' not in medal.keys()):
            if (medal['LevelNumber'] == 30) and not kc_medals_move_from_green_has_been_appended:
                kc_medals.extend(kc_medals_move_from_green)
                kc_medals_move_from_green_has_been_appended = True
            
            kc_medals.append(medal)
            continue

        # Om Grön och inte 25 m fjäril, skippa
        if (medal['LevelNumber'] == 10) and (medal['EventName'] != '25m Butterfly'):
            continue

        # Om Grön och 25 m fjäril, sätt till KC Gul och beräkna medalj
        if (medal['LevelNumber'] == 10) and (medal['EventName'] == '25m Butterfly'):
            medal['LevelNumber'] = 20
            medal['LevelName'] = 'Gula utmanaren'
            medal = set_kc_medal(medal, swimmer)
            kc_medals_move_from_green.append(medal)
            continue

        # Om Gul och inte i KC Gul, radera
        if (medal['LevelNumber'] == 20) and (medal['EventName'] not in events_yellow.keys()):
            continue

        # Om Gul och i KC Gul, beräkna medalj
        if (medal['LevelNumber'] == 20) and (medal['EventName'] in events_yellow.keys()):
            medal = set_kc_medal(medal, swimmer)
            kc_medals.append(medal)
            continue

        # Om Röd och inte i KC Röd, radera
        if (medal['LevelNumber'] == 30) and (medal['EventName'] not in events_red.keys()):
            continue

        # Om Röd och i KC Röd, beräkna medalj
        if (medal['LevelNumber'] == 30) and (medal['EventName'] in events_red.keys()):
            medal = set_kc_medal(medal, swimmer)
            kc_medals.append(medal)
            continue

    swimmer['Medals'] = kc_medals

In [130]:
def set_swimmer_level_completion_medal(swimmer, level):
    has_completed = True
    latest_received_medal_date = '0001-01-01T00:00:00'
    completion_medal = None

    for medal in swimmer['Medals']:
        if medal['EventId'] == 0 and medal['LevelNumber'] == level:
            completion_medal = medal
            continue

        if medal['LevelNumber'] == level and medal['HasReceived'] == False:
            has_completed = False

        if medal['LevelNumber'] == level and medal['HasReceived'] == True:
            date_received = datetime.datetime.strptime(medal['DateReceived'][:10], '%Y-%m-%d').date()
            date_latest_received_medal_date = datetime.datetime.strptime(latest_received_medal_date[:10], '%Y-%m-%d').date()
            if date_received > date_latest_received_medal_date:
                latest_received_medal_date = medal['DateReceived']

    completion_medal['HasReceived'] = has_completed
    if has_completed:
        completion_medal['DateReceived'] = latest_received_medal_date


In [131]:
for swimmer in swimmers:
    set_swimmer_medals(swimmer)
    set_swimmer_level_completion_medal(swimmer, 20)
    set_swimmer_level_completion_medal(swimmer, 30)

json_data['Swimmers'] = swimmers

with open('../output/Simmarhistorik-KC.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)